<div><img style="float: left; padding-right: 3em;" src="https://avatars.githubusercontent.com/u/19476722" width="150" /><div/>

# Earth Data Science Coding Challenge!
Before we get started, make sure to read or review the guidelines below. These will help make sure that your code is **readable** and **reproducible**. 

## Don't get **caught** by these Jupyter notebook gotchas

<img src="https://miro.medium.com/v2/resize:fit:4800/format:webp/1*o0HleR7BSe8W-pTnmucqHA.jpeg" width=300 style="padding: 1em; border-style: solid; border-color: grey;" />

  > *Image source: https://alaskausfws.medium.com/whats-big-and-brown-and-loves-salmon-e1803579ee36*

These are the most common issues that will keep you from getting started and delay your code review:

1. When you try to run some code on GitHub Codespaces, you may be prompted to select a **kernel**.
   * The **kernel** refers to the version of Python you are using
   * You should use the **base** kernel, which should be the default option. 
   * You can also use the `Select Kernel` menu in the upper right to select the **base** kernel
2. Before you commit your work, make sure it runs **reproducibly** by clicking:
   1. `Restart` (this button won't appear until you've run some code), then
   2. `Run All`

## Check your code to make sure it's clean and easy to read

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSO1w9WrbwbuMLN14IezH-iq2HEGwO3JDvmo5Y_hQIy7k-Xo2gZH-mP2GUIG6RFWL04X1k&usqp=CAU" height=200 />

* Format all cells prior to submitting (right click on your code).
* Use expressive names for variables so you or the reader knows what they are. 
* Use comments to explain your code -- e.g. 
  ```python
  # This is a comment, it starts with a hash sign
  ```

## Label and describe your plots

![Source: https://xkcd.com/833](https://imgs.xkcd.com/comics/convincing.png)

Make sure each plot has:
  * A title that explains where and when the data are from
  * x- and y- axis labels with **units** where appropriate
  * A legend where appropriate


## Icons: how to use this notebook
We use the following icons to let you know when you need to change something to complete the challenge:
  * &#128187; means you need to write or edit some code.
  
  * &#128214;  indicates recommended reading
  
  * &#9998; marks written responses to questions
  
  * &#127798; is an optional extra challenge
  

---

# Introduction to Multispectral Remote Sensing Data: Urban Green Space

For this assignment, you will visualize and quantify differences in vegetation health by neighborhood in Chicago, IL.

We will be developing this code over several weeks in order to practice writing **modular** code. Last week, you should have:
1. Selected two neighborhoods
2. FOR EACH neighborhood:
   1. Downloaded NAIP multispectral data for the neighborhood
   2. Calculated NDVI
   3. Calculated and save summary statistics of to a file

This week, you will:
1. Add **caching** and **garbage collection** to your analysis from the previous week. This will make sure you are making effective use of your disk space and internet connection.
2. Modularize the workflow using **functions** and/or **classes**
3. Run the workflow for all the Chicago neighborhoods and create a cloropleth plot using the summary statistics you calculated.

You should also create a portofolio piece focusing on a different city

## STEP 1: Get set up

### Package imports
Use the cell below to import the packages you need in the rest of the notebook (and **ONLY** the packages you need in the rest of the notebook).

You may also want to use this cell to define the **earth analytics data directory** and make sure it exists.

In [1]:
import os
import shutil
from glob import glob

import earthpy as et
import earthpy.earthexplorer as etee
import geopandas as gpd 
import geoviews as gv
import holoviews as hv
import numpy as np
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rxrmerge 

# Set up directories
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
denver_dir = os.path.join(data_dir, 'denver-neighborhoods')
ndvi_dir = os.path.join(data_dir, 'denver-green-space', 'processed')

for a_dir in [denver_dir, ndvi_dir]:
    if not os.path.exists(a_dir):
        os.makedirs(a_dir)

## STEP 2: Area of Interest

### Select a small number of neighborhoods to test your code and loops

In the cell below, download **and cache** a shapefile of the City of Chicago neighborhoods from [the City of Chicago Data Portal](https://data.cityofchicago.org/).

To cache downloads and calculations, you will need to use a **conditional statement**, like the following example code where `condition` is some boolean value you computed:

```python
if condition:
    do_something()
```

Note that, like for `for` loops, conditionals use **indentation** to determine what happens only when the condition is `True` and what happens no matter what.

Conditional statements can also have multiple parts, although you won't need that for this first caching step:

```python
if condition1:
    do_something()
elif condition2:
    do_something_else()
else:
    do_yet_another_thing()
```

YOUR TASK:
1. IF you don't have a City of Chicago neighborhood file saved already:
   1. Download and open up the shapefile
   2. Save it to a file using the `.to_file()` method of `GeoDataFrame`s (or some other method from earlier in the semester)
2. Load in the City of Chicago dataset from a file
3. Check that your caching is working. One way to do this is to make sure that you print something to indicate when the download is happening.

In [15]:
denver_path = os.path.join(denver_dir, 'denver_neighborhoods.shp')
if not os.path.exists(denver_path):
    denver_url = ("https://www.denvergov.org/media/gis/DataCatalog/"
              "census_neighborhood_demographics_2010/shape/census"
              "_neighborhood_demographics_2010.zip"
)
    gpd.read_file(denver_url).to_file(denver_path)
denver_gdf = gpd.read_file(denver_path).set_index('NBRHD_NAME')


neigh_gdf = (
    denver_gdf
    .loc[['Bear Valley']]
)
neigh_gdf

,NBHD_ID,POPULATION,HISPANIC_2,WHITE_2010,BLACK_2010,NATIVEAM_2,ASIAN_2010,HAWPACIS_2,OTHER_2010,TWO_OR_MOR,...,RENTED_A_2,RENTED_A_3,RENTED_A_4,RENTED_A_5,RENTED_A_6,RENTED_A_7,RENTED_A_8,SHAPE_Leng,SHAPE_Area,geometry
NBRHD_NAME,,,,,,,,,,,,,,,,,,,,,
Bear Valley,6,8889.0,3543.0,4673.0,129.0,63.0,335.0,9.0,17.0,120.0,...,299.0,272.0,94.0,50.0,70.0,25.0,7.0,0.111212,0.000368,"POLYGON ((-105.05323 39.66778, -105.05325 39.6..."


## STEP 3: Download and process raster data

You should have three loops from last week. Convert the operations from each loop into a **function**, starting with the following sample code:

```python
def download_neighborhood_data(name, geometry, start, end)
    """
    Download NAIP raster for a given geometry, start date, and end date

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    <Put your code here>
    return downloader

for neighborhood_name, details in neigh_gdf.interrows():
    download_neighborhood_data(neighborhood_name, details.geometry)

```
One important step of writing function is identifying the **Parameters** and **Returns**. In this case, I have done this for you; for later functions you will need to do this yourself. One way to identify the Parameters is to identify each object or variable used in the code (note that this does not usually include imported classes and functions). 

I am also supplying you with a **docstring** that explains the Parameters and Returns, and specifies their types. Update the docstring if you decide to do something different for your function. When writing docstrings, please follow the [numpy docstring styleguide](https://numpydoc.readthedocs.io/en/latest/format.html#sections)

YOUR TASK:

1. Replace `<Put your code here>` with the download code from last week
2. Open up your summary statistics file, if it exists.
3. Add a **conditional** to your code so that it will skip this download if the summary statistics **already exist** in your summary statistics file!
   
    > HINT: I did this using the `pass` statement, which moves on to the next iteration of the loop. This way you can test if the statistics **do** exist in the file, rather than whether they **do not**. However, there are lots of ways to do this -- do what makes sense to you!
    
4. Test that the code still works for the two-neighborhood `GeoDataFrame`. You should also check that the caching is working (although you may need to wait until you have saved some statistics to do this!)

In [16]:
def download_neighborhood_data(name, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date
    Downloads data from the National Agricultural Imagery Program (NAIP)
    given a spatial and temporal extent. 
    
    <U.S. Department of Agriculture, 2023, National Agriculture Imagery
     Program (NAIP) Data (ver. 2019, published 20191002), 
     accessed October 23, 2019 at URL [https://naip-usdaonline.hub.
     arcgis.com/search?collection=Dataset] >.

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    
    print(f' \nNeighborhood name: {name}')
    bbox = etee.BBox(*geometry.bounds)
    naip_downloader = etee.EarthExplorerDownloader(
        dataset="NAIP", 
        label=name.lower().replace(' ','-'), 
        bbox=bbox,
        start='2015-01-01', 
        end='2015-12-31',
        store_credential=True)
    naip_downloader.submit_download_request()
    naip_downloader.download(override=False)
    return naip_downloader

ndvi_stats_path = os.path.join(ndvi_dir,'neighborhood-ndvi-stats.csv')
if os.path.exists(ndvi_stats_path):
    print('Reading in NDVI statistics file...')
    ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col='neighborhood')
else: 
  print('NDVI statistics file does not exisit')
  ndvi_stats_df = pd.DataFrame()

for neighborhood_name, details in neigh_gdf.iterrows():
    if neighborhood_name in ndvi_stats_df.index:
       print('Neighborhood stats have already been calculated. Skipping')
       continue
    
    downloader = download_neighborhood_data(
       neighborhood_name, details.geometry, '2015-01-01','2015-12-31')

Reading in NDVI statistics file...
 
Neighborhood name: Bear Valley
Login Successful.
Searching datasets...
Using dataset alias: naip
Searching scenes...
Found 2 scenes
4 products found.
Downloads staging...
/home/jovyan/earth-analytics/data/bear-valley/M_3910524_SE_13_1_20150912.zip
/home/jovyan/earth-analytics/data/bear-valley/M_3910524_SE_13_1_20150912.zip


YOUR TASK: 

1. Write a function for the loop that loads and merges the arrays.
2. Document your function with a docstring
3. Check that your function works for the Lincoln Park neighborhood

# NOTE: When I try to download for more than one location...I get an error - bad zip file! 

In [19]:
def load_and_merge_arrays(name):
    """
   load in and merge downloaded arrays

    Parameters
    ==========
    name : str
      The name used to label the download

    Returns
    =======
    merged_da: rxr.DataArray
        DataArray with the merged data
    """
    
    print(f'\nNeighborhood name: {name}')
    data_path = os.path.join(
        et.io.HOME, et.io.DATA_NAME, 
        name.lower().replace(' ', '-'))
    tif_paths = glob(os.path.join(data_path, '*.tif'))
    das = [rxr.open_rasterio(tp, masked = True) for tp in tif_paths]
    merged_da = rxrmerge.merge_arrays(das)
    return merged_da

merged_da = load_and_merge_arrays('Bear Valley')
merged_da


Neighborhood name: Bear Valley


Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
/opt/conda/lib/python3.10/site-packages/rioxarray/raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


<xarray.DataArray (band: 4, y: 7580, x: 11350)>
array([[[103., 138., 134., ...,   0.,   0.,   0.],
        [118., 143., 127., ...,   0.,   0.,   0.],
        [139., 156., 138., ...,   0.,   0.,   0.],
        ...,
        [141., 149., 151., ...,  69.,  73.,  76.],
        [140., 149., 151., ...,  66.,  67.,  71.],
        [140., 145., 150., ...,  65.,  63.,  70.]],

       [[113., 138., 129., ...,   0.,   0.,   0.],
        [120., 141., 125., ...,   0.,   0.,   0.],
        [131., 145., 133., ...,   0.,   0.,   0.],
        ...,
        [135., 142., 144., ...,  95., 102., 103.],
        [134., 141., 142., ...,  94.,  97., 100.],
        [131., 139., 139., ...,  92.,  92.,  95.]],

       [[ 87., 111., 127., ...,   0.,   0.,   0.],
        [ 96., 120., 124., ...,   0.,   0.,   0.],
        [106., 125., 123., ...,   0.,   0.,   0.],
        ...,
        [131., 135., 135., ...,  70.,  72.,  75.],
        [127., 135., 135., ...,  67.,  71.,  73.],
        [129., 135., 134., ...,  71.,  69.,  72.]],

       [[170., 150.,  96., ...,   0.,   0.,   0.],
        [162., 128.,  88., ...,   0.,   0.,   0.],
        [139., 123.,  93., ...,   0.,   0.,   0.],
        ...,
        [ 70.,  77.,  78., ..., 188., 190., 191.],
        [ 69.,  74.,  77., ..., 186., 187., 189.],
        [ 68.,  71.,  77., ..., 185., 185., 186.]]], dtype=float32)
Coordinates:
  * x            (x) float64 4.89e+05 4.89e+05 4.89e+05 ... 5.003e+05 5.003e+05
  * y            (y) float64 4.393e+06 4.393e+06 ... 4.386e+06 4.386e+06
  * band         (band) int64 1 2 3 4
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:             Area
    TIFFTAG_IMAGEDESCRIPTION:  OrthoVista
    TIFFTAG_RESOLUTIONUNIT:    1 (unitless)
    TIFFTAG_SOFTWARE:          Trimble Germany GmbH
    TIFFTAG_XRESOLUTION:       0
    TIFFTAG_YRESOLUTION:       0
    scale_factor:              1.0
    add_offset:                0.0

YOUR TASK:

1. Write a function that computes the NDVI summary statistics and adds them to the statistics file (if the statistics are not already present)
    > HINT: use `mode='a'` to *append* a line to the file instead of writing over existing content
    
2. Document your function with a docstring
3. Check that your function works for the Lincoln Park Neighborhood

In [20]:
def calculate_ndvi_statistics(gdf, da, stats_path, override=False):
    """
   Calculate NDVI, then summarize and save statistics 

   Uses downloaded National Agriculture Imagery Program (NAIP) data
   < U.S. Department of Agriculture, 2023, National Agriculture Imagery
     Program (NAIP) Data (ver. 2019, published 20191002), 
     accessed October 23, 2019 at URL [https://naip-usdaonline.hub.
     arcgis.com/search?collection=Dataset] >.

    Parameters
    ==========
    gdf : gpd.GeoDataFrame
        One row with the neighborhood and boundary
    da : rxr.DataArray
        Multispectral NAIP raster data
    stats_path: pathlike 
        The path to the statistics file to save results
    """

    name = str(gdf.index[0])
    print(f'Neighborhood name: {name}')

    file_is_empty = True
    if os.path.exists(stats_path):
        print('Stats file exists.')
        stats_df = pd.read_csv(stats_path)
        file_is_empty = len(stats_df) == 0
        print(f'Stats file is empty? {file_is_empty}')

        if not file_is_empty:
            print(name)
            print(list(stats_df.neighborhood))
            print(name in list(stats_df.neighborhood))
            if (name in list(stats_df.neighborhood)) and (not override):
                print('Stats already calculated. Skipping...')   
                return 

    reprojected_gdf = gdf.to_crs(da.rio.crs)
    
    naip_crop_da = da.rio.clip_box(*reprojected_gdf.total_bounds)
    naip_da = naip_crop_da.rio.clip(reprojected_gdf.geometry)
    
    ndvi_da = (
        (da.sel(band=4) - da.sel(band=1))
        /(da.sel(band=4) + da.sel(band=1))
    )
    mode = 'w' if file_is_empty else 'a'

    pd.DataFrame(dict(
        neighborhood=['name'],
        ndvi_25pctl=[np.nanpercentile(ndvi_da, 25)],
        ndvi_mean=[float(ndvi_da.mean())]
    )).to_csv(stats_path, mode = mode, header=file_is_empty, index=False)


calculate_ndvi_statistics(
    denver_gdf.loc[["Bear Valley"]], merged_da, ndvi_stats_path)

Neighborhood name: Bear Valley
Stats file exists.
Stats file is empty? False
Bear Valley
['name']
False


Putting in all together... YOUR TASK:

1. Create a loop. Start off with just the two neighborhood `GeoDataFrame`.
2. Run each of your functions in the loop, checking that they work. **MAKE SURE YOU INCLUDE CACHING CODE!**
3. Write a line of code at the end of your loop to **delete the raster data files** once you have saved the statistics you want, checking that it works. Use the `shutil.rmtree()` function.
4. Replace the two neighborhood `GeoDataFrame` with the full Chicago `GeoDataFrame`

In [24]:
ndvi_stats_path = os.path.join(ndvi_dir,'neighborhood-ndvi-stats.csv')


for neighborhood_name, details in neigh_gdf.iterrows():
    if not os.path.exists(ndvi_stats_path):
        print('NDVI statistics file does not exisit...')
        ndvi_stats_df = pd.DataFrame()

    else: 
        ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")

    if neighborhood_name in ndvi_stats_df.index:
       print('Neighborhood stats have already been calculated. Skipping')
       continue

    download_neighborhood_data(
        neighborhood_name, details.geometry, '2015-01-01','2015-12-31')
    merged_da = load_and_merge_arrays(neighborhood_name)
    calculate_ndvi_statistics(
        denver_gdf.loc[[neighborhood_name]], merged_da, ndvi_stats_path)
    
    shutil.rmtree(downloader.data_dir)


 
Neighborhood name: Bear Valley
Login Successful.
Searching datasets...
Using dataset alias: naip
Searching scenes...
Found 2 scenes
4 products found.
Downloads staging...
/home/jovyan/earth-analytics/data/bear-valley/M_3910524_SE_13_1_20150912.zip
Saving download: M_3910524_SE_13_1_20150912
/home/jovyan/earth-analytics/data/bear-valley/M_3910524_SE_13_1_20150912.zip

Neighborhood name: Bear Valley
Neighborhood name: Bear Valley
Stats file exists.
Stats file is empty? False
Bear Valley
['name', 'name', 'name']
False


# Repeat this process for all Denver neighborhoods...or at least attempt to...if downloader cooperates!

In [25]:
ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')

for neighborhood_name, details in denver_gdf.iterrows():
    if not os.path.exists(ndvi_stats_path):
        print('NDVI statistics file does not exist...')
        ndvi_stats_df = pd.DataFrame()
    else:
        ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")

    if neighborhood_name in ndvi_stats_df.index:
        print(f'Neighborhood name: {neighborhood_name}')
        print('Neighborhood stats have already been calculated. Skipping.')
        continue

    try:
        downloader = download_neighborhood_data(
            neighborhood_name, details.geometry, '2021-01-01', '2021-12-31')
        
    except (ValueError, TypeError):
        print('Download not available. Try different date range.')
        try:
            downloader = download_neighborhood_data(
                neighborhood_name, details.geometry, '2022-01-01', '2022-12-31')
        except (ValueError, TypeError):
            print('Data cannot be found. Skipping neighborhood.')
            continue

    merged_da = load_and_merge_arrays(neighborhood_name)
    calculate_ndvi_statistics(
        denver_gdf.loc[[neighborhood_name]], merged_da, ndvi_stats_path)

    shutil.rmtree(downloader.data_dir)

 
Neighborhood name: Hampden
Login Successful.
Searching datasets...
Using dataset alias: naip
Searching scenes...
Found 2 scenes
4 products found.
Downloads staging...

 2 downloads queued but not yet available. Waiting for 30 seconds.

/home/jovyan/earth-analytics/data/hampden/M_3910417_SE_13_1_20150910.zip
Saving download: M_3910417_SE_13_1_20150910
/home/jovyan/earth-analytics/data/hampden/M_3910418_SW_13_1_20150910.zip
Saving download: M_3910418_SW_13_1_20150910
/home/jovyan/earth-analytics/data/hampden/M_3910418_SW_13_1_20150910.zip
/home/jovyan/earth-analytics/data/hampden/M_3910417_SE_13_1_20150910.zip

Neighborhood name: Hampden
Neighborhood name: Hampden
Stats file exists.
Stats file is empty? False
Hampden
['name', 'name', 'name', 'name']
False
 
Neighborhood name: Baker
Login Successful.
Searching datasets...
Using dataset alias: naip
Searching scenes...
Found 2 scenes
4 products found.
Downloads staging...
/home/jovyan/earth-analytics/data/baker/M_3910417_NW_13_1_20150910.

BadZipFile: File is not a zip file

## STEP 4: Plot

YOUR TASK:
1. Join your `GeoDataFrame` of Chicago neighborhoods with your NDVI statistics `DataFrame`
2. Create a Chloropleth plot using one of the statistics for the color scale
3. Write a plot headline and description.

In [22]:
ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
chloropleth = gv.tile_sources.StamenToner * gv.Polygons(
    denver_gdf.join(ndvi_stats_df, how='left'),
    vdims=['ndvi_mean']
)
hv.save(chloropleth, 'chloropleth.html')

In [23]:
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
denver_dir = os.path.join(data_dir, 'denver-neighborhoods')
ndvi_dir = os.path.join(data_dir, 'denver-green-space', 'processed')

for a_dir in [denver_dir, ndvi_dir]:
    if not os.path.exists(a_dir):
        os.makedirs(a_dir)